In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
yrs=[1983,1984,1985]
arr=[]
links=[]
for yr in yrs:
    html=requests.get("https://www.profootballarchives.com/"+str(yr)+"usfl.html")
    soup=BeautifulSoup(html.text)
    stats=soup.find("div",{"id":"stats"})
    table=stats.find_all("table")[0]
    for a in table.find_all("a"):
        links.append(a.get("href"))
        url="https://www.profootballarchives.com/"+str(a.get("href"))
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[4]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(yr)
            arr.append(h)
yrs=[2022]
links=[]
html=requests.get("https://www.profootballarchives.com/2022usflbir.html")
soup=BeautifulSoup(html.text)
table=soup.find_all("table")[3]
for a in table.find_all("a"):
    if '<a></a>' not in str(a):
        links.append(a.get("href"))
for link in links:
    for yr in yrs:
        
        link=str(link)
        if '83' in link:
            link=link.split("83")
            link=link[1]
        if '84' in link:
            link=link.split("84")
            link=link[1]
        if '85' in link:
            link=link.split("85")
            link=link[1]
        link=link
        url="https://www.profootballarchives.com/"+link
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[3]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(2022)
            arr.append(h)
df=pd.concat([pd.DataFrame(arr)])
df.columns=['Day','Date','VenueType','team2','score','result','OT','Location','Venue','Attendance','team1','season']
df=df.astype({'VenueType':'string'})

df=df[df.Date.notna()]
df=df[df.score.notna()]
df['score2']=0
df['score1']=0
for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    df.at[row.Index,'score1']=s1
    df.at[row.Index,'score2']=s2
df['Date']=df['Date'].str.replace("/","")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,"Date":"string"})  
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)


KeyboardInterrupt: 

In [1]:
f

NameError: name 'f' is not defined

In [3]:
import math
import numpy as np
import pandas as pd
df=pd.read_csv("https://github.com/easella/usfl/raw/main/usflscores.csv")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string"})
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
hfa=[]
w=0

df.sort_values(by='Date',inplace=True)
l=0
gh={}
gh['g']=0
class Elo:
  global winnerHome
  def __init__(self,k,g=1,homefield = 182.5):
    self.ratingDict   = {}  
    self.k        = k
    self.g        = g
  def addPlayer(self,name,rating = 1500):
    self.ratingDict[name] = rating
  def gameOver(self, winner, loser, winnerHome,neutral,wins,loses):
    homef=gh['g']
    wb=0
    lb=0
    if winnerHome and neutral!='N':
      result = self.expectResult(self.ratingDict[winner] + homef, self.ratingDict[loser])
    if winnerHome==False and neutral!='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser] + homef)
    if neutral=='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser])
    wins=float(wins)
    loses=float(loses)
    if wins==loses:
        tie=True
    else:
        tie=False
    if tie:
        mult=(math.log(0+1))*(2.2/1*0.001+2.2)
        shift=(self.k*mult)*(0.5 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
    if tie !=True:
        win=eloLeague.ratingDict[winner]
        lose=eloLeague.ratingDict[loser]
        if neutral=='N':
          win=(eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome and neutral!='N':
          win=(homef+eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome!=True and neutral!='N':
          lose=(homef+eloLeague.ratingDict[loser])
          win=(eloLeague.ratingDict[winner])
        global zy
        if(win>lose):
          zy=(win-lose)
        if(win==lose):
          zy=(win-lose)
        if(win<lose):
          zy=(win-lose)
        mult=(math.log((wins-loses)+1))*(2.2/(zy)*0.001+2.2)
        shift=(self.k*mult)*(1 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
  def expectResult(self, p1, p2):
        global exp
        exp = (p2-p1)/400
        n2=10
        j=exp
        o=1/((n2**(j))+1)
        return float(o)
teams=set(df.team1.tolist()+df.team2.tolist())

eloLeague=Elo(k=25)
currSeason=1983
q=pd.read_html("https://en.wikipedia.org/wiki/2023_USFL_season")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[10]':'Team'})
q=q.rename(columns={'Home stadium':'Venue'})
venues={}
venues[2023]={}
for row in q.itertuples():
    l=list(row)
    team=str(l[1])
    team=team.replace("[a]","")
    team=team.replace(" ","")
    team=team.replace("MemphisShowboats","TampaBayBandits")
    venues[2023][team]=row.Venue


    
q=pd.read_html("https://en.wikipedia.org/wiki/United_States_Football_League_(2022)#Teams")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[72]':'Team'})
q=q.rename(columns={'Stadium':'Venue'})
venues[2022]={}
for row in q.itertuples():
    l=list(row)
    team=str(l[1])
    team=team.replace("[a]","")
    team=team.replace(" ","")
    team=team.replace("MemphisShowboats","TampaBayBandits")

    venues[2022][team]=row.Venue
t=pd.read_csv("https://ontheroadtovote.com/ncaab/usflscores.txt")
df=pd.concat([df,t])
for row in df.itertuples():
    if row.season>2021:
        if venues[row.season][row.team2]==venues[row.season][row.team1]:
            df.at[row.Index,'VenueType']='N'
df=df.drop(['Unnamed: 0'], axis=1)
df=df.drop_duplicates()
hfa=[]
df['score2']=0
df['score1']=0

for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    if row.season==2023:
        df.at[row.Index,'score1']=s2
        df.at[row.Index,'score2']=s1
    if row.season!=2023:
        df.at[row.Index,'score1']=s2
        df.at[row.Index,'score2']=s1
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string",'VenueType':'string'})
n=df[df.VenueType!='N']
hfaz=[]
for row in n.itertuples():
    if row.VenueType=='A':
        hfaz.append(row.score1-row.score2)
    else:
        hfaz.append(row.score2-row.score1)
homef=round(np.mean(hfaz))
for team in teams:
    eloLeague.addPlayer(team,rating=1500)
gh['g']=round(np.mean(hfaz)*25)
l=0
w=0
df=df.replace("MemphisShowboats","TampaBayBandits")
msqe={}
for i in range(10,50):
    msqe[i]=[]
for row in df.itertuples():
    if float(row.season)>currSeason:
        season=float(row.season)
        
                
       
            
       
        
        
        for key in eloLeague.ratingDict.keys():
            
            
                
            pre=eloLeague.ratingDict[key]
            if eloLeague.ratingDict[key]!=1500:
                eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1505*(1/3))
            af=eloLeague.ratingDict[key]
    
                
        currSeason+=1
    if row.score1>row.score2 or row.score1==row.score2:
        hfa=0
        if row.VenueType!='N':
            hfa=float(gh['g'])
        if row.VenueType=='A':

            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
            
        if row.VenueType==('H' or 'N'):
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=False
        eloLeague.gameOver(row.team1,row.team2,winnerhome,row.VenueType,row.score1,row.score2)
    if row.score1<row.score2:
        hfa=0
        if row.VenueType!='N':
            hfa=gh['g']
        if row.VenueType=='A':
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=False
        if row.VenueType=='H':
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=True
        if row.VenueType=='N':
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
        eloLeague.gameOver(row.team2,row.team1,winnerhome,row.VenueType,row.score2,row.score1)
    from sklearn.metrics import mean_squared_error
    if row.VenueType =='N' or row.VenueType=='H':
        yt = [row.score2-row.score1]
        pred=(eloLeague.ratingDict[row.team2]+hfa)-eloLeague.ratingDict[row.team1]
    else:
        yt = [row.score1-row.score2]
        pred=(eloLeague.ratingDict[row.team1]+hfa)-eloLeague.ratingDict[row.team2]
    for i in range(10,50):
        y_pred=[round((pred)/i)]
        msqe[i].append(mean_squared_error(yt, y_pred))
css=df[df.season==2023]
if len(css)==0:
    lsdf=df[df.season<2023]
    ls=set(lsdf.team1.tolist()+lsdf.team2.tolist())
    csdf=df[df.season<2023]
    cs=set(csdf.team1.tolist()+csdf.team2.tolist()) 
    for key in eloLeague.ratingDict.keys():
            
        if(key  in ls and key in cs):
            pre=eloLeague.ratingDict[key]
            eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1500*(1/3))
            af=eloLeague.ratingDict[key]
            print(pre,af,key)
ms={}
for i in range(10,50):
    ms[i]=0
for key in msqe.keys():
    ms[key]=np.mean(msqe[key])
mas={k: v for k, v in sorted(ms.items(), key=lambda item: item[1],reverse=False)[:1]}
for key in mas.keys():
    print("best value for spreads:"+str(key))    

1437.594827823623 1458.3965518824152 MichiganPanthers
1505.000026750322 1503.3333511668811 DenverGold
1505.0000547262541 1503.3333698175027 WashingtonFederals
1505.0000002129775 1503.3333334753183 ArizonaOutlaws
1504.999981663073 1503.3333211087152 BostonBreakers
1505.0000996853662 1503.3333997902441 LosAngelesExpress
1504.9998767162951 1503.3332511441968 BaltimoreStars
1416.4420834731498 1444.294722315433 PhiladelphiaStars
1505.0000375042796 1503.3333583361864 OrlandoRenegades
1505.000054516032 1503.3333696773545 SanAntonioGunslingers
1444.5204979041966 1463.0136652694644 BirminghamStallions
1433.7863322608068 1455.857554840538 HoustonGamblers
1504.9999490612668 1503.3332993741778 ArizonaWranglers
1505.0000454905128 1503.3333636603418 PortlandBreakers
1602.6837062335896 1568.4558041557264 PittsburghMaulers
1505.0000333534736 1503.3333555689824 ChicagoBlitz
1504.9999669335098 1503.3333112890064 JacksonvilleBulls
1578.5076556281329 1552.338437085422 TampaBayBandits
1526.7346040077102 15

In [27]:
q=pd.read_html("https://en.wikipedia.org/wiki/United_States_Football_League_(2022)#Teams")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[72]':'Team'})
q=q.rename(columns={'Stadium':'Venue'})
q

,Team[40],Based City/State,Venue,Capacity,First season,Head coach,Unnamed: 6_level_0,Unnamed: 7_level_0
0,Michigan Panthers,"Detroit, Michigan",Ford Field,65000,2022,Mike Nolan,NaN,NaN
1,New Jersey Generals,"East Rutherford, New Jersey",Tom Benson Hall of Fame Stadium,23000,2022,Mike Riley,NaN,NaN
2,Philadelphia Stars,"Philadelphia, Pennsylvania",Ford Field,65000,2022,Bart Andrus,NaN,NaN
3,Pittsburgh Maulers,"Pittsburgh, Pennsylvania",Tom Benson Hall of Fame Stadium,23000,2022,Ray Horton[41],NaN,NaN
4,South Division,South Division,South Division,South Division,South Division,South Division,South Division,South Division
5,Birmingham Stallions,"Birmingham, Alabama",Protective Stadium,47100,2022,Skip Holtz,NaN,NaN
6,Houston Gamblers,"Houston, Texas",Simmons Bank Liberty Stadium,58325,2022,Curtis Johnson,NaN,NaN
7,Memphis Showboats,"Memphis, Tennessee",Simmons Bank Liberty Stadium,58325,2023,Todd Haley,NaN,NaN
8,New Orleans Breakers,"New Orleans, Louisiana",Protective Stadium,47100,2022,John DeFilippo,NaN,NaN


In [95]:
ds={}
for i in range(1983,1986):
    ds[i]={}
for i in range(1983,1986):
    
    dsdf=pd.read_html("https://www.profootballarchives.com/"+str(i)+"usfl.html")[2]
    
    dsdf.columns=dsdf.columns.droplevel(1)
    dsdf.columns=['Team','W','L','T','PCT','PF','PA','w','q','e','t','k','j','l','m']
    dsdf=dsdf.astype({'Team':'string','PCT':'string'})
    k=0
    for row in dsdf.itertuples():
        if k==0:
            div='ggg'
            ds[i][div]={}
        if 'PCT' in str(row.PCT):
            print('yes')
            if str(row.Team) not in str(ds[i]):
                print('dd')
                print(row.Team)
                if k==0:
                    div='ggg'
                else:
                    div=row.Team
                ds[i][div]={}
        else:
            tea=str(row.Team)
            tea=tea.replace("*","")
            tea=tea.replace(" ","")
            if 'tea' not in str(ds[i]):
                ds[i][div][tea]=0
            else:
                ds[i][div][tea]=eloLeague.ratingDict[tea]
        k+=1
            
                
    

yes
dd
Central Division
yes
dd
Pacific Division
yes
dd
Southern Division
yes
dd
Central Division
yes
dd
Pacific Division
yes
dd
Western Conference


In [97]:
ds

{1983: {'ggg': {'PhiladelphiaStars': 0,
   'BostonBreakers': 0,
   'NewJerseyGenerals': 0,
   'WashingtonFederals': 0},
  'Central Division': {'MichiganPanthers': 0,
   'ChicagoBlitz': 0,
   'TampaBayBandits': 0,
   'BirminghamStallions': 0},
  'Pacific Division': {'OaklandInvaders': 0,
   'LosAngelesExpress': 0,
   'DenverGold': 0,
   'ArizonaWranglers': 0}},
 1984: {'ggg': {'PhiladelphiaStars': 0,
   'NewJerseyGenerals': 0,
   'WashingtonFederals': 0,
   'PittsburghMaulers': 0},
  'Southern Division': {'BirminghamStallions': 0,
   'TampaBayBandits': 0,
   'NewOrleansBreakers': 0,
   'MemphisShowboats': 0,
   'JacksonvilleBulls': 0,
   'WesternConference': 0},
  'Central Division': {'HoustonGamblers': 0,
   'MichiganPanthers': 0,
   'SanAntonioGunslingers': 0,
   'OklahomaOutlaws': 0,
   'ChicagoBlitz': 0},
  'Pacific Division': {'LosAngelesExpress': 0,
   'ArizonaWranglers': 0,
   'DenverGold': 0,
   'OaklandInvaders': 0}},
 1985: {'ggg': {'BirminghamStallions': 0,
   'NewJerseyGener

In [94]:
str(row.Team) not in str(ds[i])

True

In [93]:
ds

{1983: {}, 1984: {}, 1985: {}}

In [61]:
"https://www.profootballarchives.com/"+str(i)+"usfl.html"

'https://www.profootballarchives.com/1985usfl.html'

In [48]:
{k: v for k, v in sorted(ms.items(), key=lambda item: item[1],reverse=False)[:1]}

{47: 239.52954048140043}

In [22]:
ds={}

In [24]:
year=input("season")
ds[str(year)]={}
if 'nd' not in str(ds[year]):
    ds[year]['nd']={}
    ds[year]['sd']={}
for team in teams:
    d=input("div")
    ds[year][d][team]={}
    
    

season 2023


KeyboardInterrupt: Interrupted by user

In [46]:
ms={}
for i in range(10,50):
    ms[i]=0
for key in msqe.keys():
    ms[key]=np.mean(msqe[key])
mas={k: v for k, v in sorted(ms.items(), key=lambda item: item[1],reverse=False)[:1]}
for key in mas.keys():
    print(key)

In [14]:
eloLeague.ratingDict['NewOrleansBreakers']+float(gh['g'])

1632.4867797884729

In [17]:
1632.4867797884729-1568.4558041557264

64.0309756327465

In [16]:
eloLeague.ratingDict['PittsburghMaulers']

1568.4558041557264

In [30]:
df[df.score1==df.score2]

,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
86,Sun,1985-03-03 00:00:00,H,BaltimoreStars,17-17,T,OT,"Oakland, CA",Oakland-Alameda County Coliseum,20495.0,OaklandInvaders,1985.0,17.0,17.0


In [28]:
df[df.team2=='ChicagoBlitz']

,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
31,Sat,1983-03-12 00:00:00,H,ChicagoBlitz,30-29,W,NaN,"Tempe, AZ",Sun Devil Stadium,28434.0,ArizonaWranglers,1983.0,30.0,29.0
59,Sun,1983-03-20 00:00:00,A,ChicagoBlitz,16-13,W,NaN,"Chicago, IL",Soldier Field,22600.0,DenverGold,1983.0,16.0,13.0
81,Sun,1983-03-27 00:00:00,A,ChicagoBlitz,14-20,L,NaN,"Chicago, IL",Soldier Field,10936.0,LosAngelesExpress,1983.0,14.0,20.0
151,Sat,1983-04-02 00:00:00,H,ChicagoBlitz,3-42,L,NaN,"Tampa, FL",Tampa Stadium,46585.0,TampaBayBandits,1983.0,3.0,42.0
122,Sun,1983-04-10 00:00:00,A,ChicagoBlitz,11-22,L,NaN,"Chicago, IL",Soldier Field,13859.0,BirminghamStallions,1983.0,11.0,22.0
224,Sun,1983-05-01 00:00:00,H,ChicagoBlitz,17-38,L,NaN,"Los Angeles, CA",Memorial Coliseum,21123.0,LosAngelesExpress,1983.0,17.0,38.0
314,Mon,1983-05-30 00:00:00,A,ChicagoBlitz,11-36,L,NaN,"Chicago, IL",Soldier Field,13952.0,ArizonaWranglers,1983.0,11.0,36.0
358,Sun,1983-06-12 00:00:00,A,ChicagoBlitz,8-31,L,NaN,"Chicago, IL",Soldier Field,21249.0,TampaBayBandits,1983.0,8.0,31.0
378,Fri,1983-06-17 00:00:00,H,ChicagoBlitz,14-29,L,NaN,"Birmingham, AL",Legion Field,22500.0,BirminghamStallions,1983.0,14.0,29.0
468,Sun,1983-07-03 00:00:00,A,ChicagoBlitz,7-31,L,NaN,"Chicago, IL",Soldier Field,12346.0,OaklandInvaders,1983.0,7.0,31.0


In [29]:
print(100/(w+l)*w)

63.57615894039735


In [32]:
l2s={k: v for k, v in sorted(eloLeague.ratingDict.items(), key=lambda item: item[1],reverse=True)}
for key in l2s.keys():
    print(key,eloLeague.ratingDict[key])

NewOrleansBreakers 1614.9176393022417
MichiganPanthers 1552.050254621962
BirminghamStallions 1541.613164963485
PittsburghMaulers 1523.358277975291
PhiladelphiaStars 1510.7062918943711
LosAngelesExpress 1505.0000664569106
OklahomaOutlaws 1505.000046158328
WashingtonFederals 1505.0000364841694
SanAntonioGunslingers 1505.0000363440213
PortlandBreakers 1505.0000303270085
OrlandoRenegades 1505.000025002853
ChicagoBlitz 1505.0000222356489
DenverGold 1505.0000178335479
ArizonaOutlaws 1505.0000001419849
BostonBreakers 1504.999987775382
JacksonvilleBulls 1504.999977955673
ArizonaWranglers 1504.9999660408444
OaklandInvaders 1504.9999378578227
BaltimoreStars 1504.9999178108633
MemphisShowboats 1500
TampaBayBandits 1489.2602008398171
NewJerseyGenerals 1442.5569029777862
HoustonGamblers 1365.5371854343243
